In [1]:
%load_ext autoreload
%autoreload 2

In [93]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from extract_feats import *
#from hydrate_tweets import *

In [10]:
df = pd.read_csv('data/training.csv')
df

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,1
15998,i feel like this was such a rude comment and i...,3


In [11]:
df_test = pd.read_csv('data/test.csv')
df_test

,text,label
0,im feeling rather rotten so im not very ambiti...,0
1,im updating my blog because i feel shitty,0
2,i never make her separate from me because i do...,0
3,i left with my bouquet of red and yellow tulip...,1
4,i was feeling a little vain when i did this one,0
...,...,...
1995,i just keep feeling like someone is being unki...,3
1996,im feeling a little cranky negative after this...,3
1997,i feel that i am useful to my people and that ...,1
1998,im feeling more comfortable with derby i feel ...,1


In [13]:
label_mapping = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear'}

In [5]:
'''
features = {'anger': 0, 'anticipation': 1, 'disgust': 2, 'fear': 3, 'joy': 4, 'sadness': 5,
            'surprise': 6, 'trust': 7, 'negative': 8, 'positive': 9, 'CC': 10, 'IN': 11, 'JJR': 12,
            'JJS': 13, 'PRP': 14, '!': 15, '?': 16}
nrc_pos_feats = extract_nrc_pos_feats(df, features)
nrc_pos_feats
'''

"\nfeatures = {'anger': 0, 'anticipation': 1, 'disgust': 2, 'fear': 3, 'joy': 4, 'sadness': 5,\n            'surprise': 6, 'trust': 7, 'negative': 8, 'positive': 9, 'CC': 10, 'IN': 11, 'JJR': 12,\n            'JJS': 13, 'PRP': 14, '!': 15, '?': 16}\nnrc_pos_feats = extract_nrc_pos_feats(df, features)\nnrc_pos_feats\n"

In [14]:
cleaned_data = cleanData(df)
cleaned_data

,text,label
0,didnt feel humili,0
1,go feel hopeless damn hope around someon care ...,0
2,im grab minut post feel greedi wrong,3
3,ever feel nostalg fireplac know still properti,2
4,feel grouchi,3
...,...,...
15995,veri brief time beanbag said anna feel like be...,0
15996,turn feel pathet still wait tabl sub teach degre,0
15997,feel strong good overal,1
15998,feel like thi wa rude comment im glad,3


In [12]:
cleaned_test = cleanData(df_test)
cleaned_test

,text,label
0,im feel rather rotten im veri ambiti right,0
1,im updat blog becaus feel shitti,0
2,never make separ becaus ever want feel like asham,0
3,left bouquet red yellow tulip arm feel slightl...,1
4,wa feel littl vain thi one,0
...,...,...
1995,keep feel like someon unkind wrong think get b...,3
1996,im feel littl cranki neg thi doctor appoint,3
1997,feel use peopl give great feel achiev,1
1998,im feel comfort derbi feel though start step s...,1


In [15]:
X_train, y_train, vectorizer = createBoWFeatureVec(cleaned_data)

In [19]:
X_test = vectorizer.transform(cleaned_test['text']).toarray()
y_test = cleaned_test.loc[:, 'label'].values

In [17]:
X_train, y_train

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([0, 0, 3, ..., 1, 3, 0]))

In [20]:
X_test, y_test

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([0, 0, 0, ..., 1, 1, 4]))

In [21]:
X2_train, y_train, vectorizer2 = createTfIdfFeatureVec(cleaned_data)

In [22]:
X2_test = vectorizer2.transform(cleaned_test['text']).toarray()
y_test = cleaned_test.loc[:, 'label'].values

In [23]:
X2_train, y_train

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([0, 0, 3, ..., 1, 3, 0]))

In [24]:
X2_test, y_test

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([0, 0, 0, ..., 1, 1, 4]))

In [55]:
clf, _ = trainLogisticRegression(X_train, X_test, y_train, y_test) #no taking into account the class imbalance

In [56]:
clf2, _ = trainLogisticRegression(X2_train, X2_test, y_train, y_test) #no taking into account the class imbalance

In [59]:
y_pred = clf.predict(X_test)

In [60]:
y2_pred = clf2.predict(X2_test)

A macro-average will compute the metric independently for each class and then take the average (hence treating all classes equally), whereas a micro-average will aggregate the contributions of all classes to compute the average metric.

In [71]:
#using BoW
average = 'macro'
print(f'Accuracy score:{accuracy_score(y_test,y_pred)}') 
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y_pred)}') 
print(f'Area Under Curve:{roc_auc_score_multiclass(y_test,y_pred)}') 
print(f'Precision score:{precision_score(y_test,y_pred,average=average)}') 
print(f'Recall score:{recall_score(y_test,y_pred,average=average)}') 
print(f'F1 score:{f1_score(y_test,y_pred,average=average)}') 

Accuracy score:0.86
Confusion Matrix:
[[520  23   7  16  11   4]
 [ 11 630  30   9   4  11]
 [  7  37 108   2   3   2]
 [ 22  10   2 231   9   1]
 [ 16   4   0  12 183   9]
 [  2   5   0   0  11  48]]
Area Under Curve:{0: 0.9270673755123181, 1: 0.922969210838226, 2: 0.8290305719820031, 3: 0.9086956521739129, 4: 0.8977839446589447, 5: 0.8566560120334681}
Precision score:0.8077555282299942
Recall score:0.8108276203336242
F1 score:0.8086121544586198


In [72]:
#using TfIdF
print(f'Accuracy score:{accuracy_score(y_test,y2_pred)}') 
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y2_pred)}') 
print(f'Area Under Curve:{roc_auc_score_multiclass(y_test,y2_pred)}')
print(f'Precision score:{precision_score(y_test,y2_pred,average=average)}') 
print(f'Recall score:{recall_score(y_test,y2_pred,average=average)}') 
print(f'F1 score:{f1_score(y_test,y2_pred,average=average)}')

Accuracy score:0.842
Confusion Matrix:
[[524  30   2  17   7   1]
 [ 11 648  23   7   2   4]
 [ 15  54  84   3   2   1]
 [ 27  22   1 219   6   0]
 [ 24  13   0  11 175   1]
 [  3  16   0   1  12  34]]
Area Under Curve:{0: 0.9227577783195604, 1: 0.9144629124286778, 2: 0.7570895637112725, 3: 0.8868774703557312, 4: 0.8824605855855856, 5: 0.7557660367898217}
Precision score:0.8324534797745402
Recall score:0.7425557377437476
F1 score:0.776803125731786


In [63]:
w={0:4, 1:8, 2:12, 3:13, 4:29, 5:34} #weights as inverse of label distribution for class imbalance 

In [64]:
clf3, _ = trainLogisticRegression_weighted(X_train, X_test, y_train, y_test, w) #taking into account the class imbalance

In [67]:
clf4, _ = trainLogisticRegression_weighted(X2_train, X2_test, y_train, y_test, w) #taking into account the class imbalance

In [76]:
y3_pred = clf3.predict(X_test)

In [77]:
y4_pred = clf4.predict(X2_test)

In [84]:
#using BoW
average = 'micro' 
print(f'Accuracy score:{accuracy_score(y_test,y3_pred)}') 
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y3_pred)}') 
print(f'Area Under Curve:{roc_auc_score_multiclass(y_test,y3_pred)}') 
print(f'Precision score:{precision_score(y_test,y3_pred,average=average)}') 
print(f'Recall score:{recall_score(y_test,y3_pred,average=average)}') 
print(f'F1 score:{f1_score(y_test,y3_pred,average=average)}') 

Accuracy score:0.8425
Confusion Matrix:
[[491  33  10  26  16   5]
 [  7 611  45  10   9  13]
 [  5  31 113   5   3   2]
 [ 14   9   2 234  15   1]
 [  9   4   0  12 186  13]
 [  1   5   0   1   9  50]]
Area Under Curve:{0: 0.909862342756711, 1: 0.9081507208026682, 2: 0.839865194948056, 3: 0.9098023715415021, 4: 0.9005389317889317, 5: 0.8699978063990473}
Precision score:0.8425
Recall score:0.8425
F1 score:0.8425


In [92]:
#using TfIdF
print(f'Accuracy score:{accuracy_score(y_test,y4_pred)}') 
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y4_pred)}')
print(f'Area Under Curve:{roc_auc_score_multiclass(y_test,y4_pred)}') 
print(f'Precision score:{precision_score(y_test,y4_pred,average=average)}') 
print(f'Recall score:{recall_score(y_test,y4_pred,average=average)}') 
print(f'F1 score:{f1_score(y_test,y4_pred,average=average)}')

Accuracy score:0.8475
Confusion Matrix:
[[482  36   7  25  25   6]
 [  4 625  37  12   7  10]
 [  2  36 109   7   3   2]
 [ 13  12   2 236  11   1]
 [  6   3   0   9 196  10]
 [  1   5   0   1  12  47]]
Area Under Curve:{0: 0.9056406841500706, 1: 0.9143912456241903, 2: 0.8302740853856428, 3: 0.9134387351778657, 4: 0.9211711711711713, 5: 0.8485631913760145}
Precision score:0.8475
Recall score:0.8475
F1 score:0.8475


In [106]:
w=compute_class_weight(class_weight = "balanced",classes = np.unique([0,1,2,3,4,5]),y = y_train)
class_weights = dict(enumerate(w.flatten(), 0))
class_weights

{0: 0.5715102157451064,
 1: 0.49732686808404825,
 2: 2.044989775051125,
 3: 1.2351397251814111,
 4: 1.3766993632765445,
 5: 4.662004662004662}

In [107]:
clf5, _ = trainLogisticRegression_weighted(X_train, X_test, y_train, y_test, class_weights) #taking into account the class imbalance using another method

In [108]:
clf6, _ = trainLogisticRegression_weighted(X2_train, X2_test, y_train, y_test, class_weights) #taking into account the class imbalance using another method

In [109]:
y5_pred = clf5.predict(X_test)

In [110]:
y6_pred = clf6.predict(X2_test)

In [111]:
#using BoW
average = 'micro' 
print(f'Accuracy score:{accuracy_score(y_test,y5_pred)}') 
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y5_pred)}') 
print(f'Area Under Curve:{roc_auc_score_multiclass(y_test,y5_pred)}') 
print(f'Precision score:{precision_score(y_test,y5_pred,average=average)}') 
print(f'Recall score:{recall_score(y_test,y5_pred,average=average)}') 
print(f'F1 score:{f1_score(y_test,y5_pred,average=average)}') 

Accuracy score:0.8525
Confusion Matrix:
[[505  17  12  24  17   6]
 [ 10 598  56  10   7  14]
 [  5  20 126   3   3   2]
 [ 18   6   4 237   9   1]
 [ 10   1   1  10 182  20]
 [  1   3   0   0   5  57]]
Area Under Curve:{0: 0.9190916489879785, 1: 0.9122081645028806, 2: 0.8764002336712001, 3: 0.9172859025032938, 4: 0.8947072072072072, 5: 0.9207013255617186}
Precision score:0.8525
Recall score:0.8525
F1 score:0.8525


In [112]:
#using TfIdF
print(f'Accuracy score:{accuracy_score(y_test,y6_pred)}') 
print(f'Confusion Matrix:\n{confusion_matrix(y_test,y6_pred)}')
print(f'Area Under Curve:{roc_auc_score_multiclass(y_test,y6_pred)}') 
print(f'Precision score:{precision_score(y_test,y6_pred,average=average)}') 
print(f'Recall score:{recall_score(y_test,y6_pred,average=average)}') 
print(f'F1 score:{f1_score(y_test,y6_pred,average=average)}')

Accuracy score:0.844
Confusion Matrix:
[[492  20  14  25  21   9]
 [ 11 583  66  12   6  17]
 [  5  14 133   3   2   2]
 [ 13   9   5 237  10   1]
 [  8   2   1  11 185  17]
 [  2   3   0   0   3  58]]
Area Under Curve:{0: 0.9096658455022142, 1: 0.9010336558339535, 2: 0.8948821224450754, 3: 0.9161264822134387, 4: 0.9011221042471043, 5: 0.9275014885149322}
Precision score:0.844
Recall score:0.844
F1 score:0.844


In [37]:
df = pd.read_csv('data/corona_tweets/corona_tweets_01.csv', header=None)
df

,0,1
0,1240727808080412673,0.357143
1,1240727808005079041,0.116071
2,1240727808340414464,-0.050000
3,1240727808629813248,-0.714286
4,1240727808617230336,0.700000
...,...,...
831322,1240861453524930562,0.000000
831323,1240861453554315265,0.140693
831324,1240861453503959043,0.000000
831325,1240861453311070209,0.000000


In [43]:
tweet_fields=['created_at'],
user_fields=['description']

text_data, author_data, timestamp_data = hydrateTweets(df, tweet_fields, user_fields)
d = {'text': text_data, 'author': author_data, 'timestamp': timestamp_data}
tweet_data = pd.DataFrame(data=d)
tweet_data

,text,author,timestamp
0,longer defend make excuse reasoning want inten...,trying use wits good evil people united change...,2020-03-19T19:52:15.000Z
1,everyone looks sick,starchild go wild ig remingtonxx,2020-03-19T19:52:15.000Z
2,good came china,,2020-03-19T19:52:15.000Z
3,corona day feels like sunday ...,,2020-03-19T19:52:15.000Z
4,close president notes seen crossed corona repl...,,2020-03-19T19:52:15.000Z
...,...,...,...
56,asked dc cop noticed since coronavirus sent pe...,ut-business marketing mastersmgmt aquinas prou...,2020-03-19T19:52:18.000Z
57,lot would actually benefit taking good look in...,,2020-03-19T19:52:18.000Z
58,close president notes seen crossed corona repl...,i'm great believer luck find harder work thoma...,2020-03-19T19:52:18.000Z
59,among things shows they're using term coronavi...,pro market pro immigrant pro gmo anti supersti...,2020-03-19T19:52:18.000Z


In [39]:
X, y = createBoWFeatureVecTweets(tweet_data['text'].tolist(), df[1].values, vectorizer)
X, y

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0]]),
 array([ 0.35714286,  0.11607143, -0.05      , ...,  0.        ,
         0.        ,  0.        ]))

In [40]:
X, y = createTfIdfFeatureVecTweets(tweet_data['text'].tolist(), df[1].values, vectorizer2)
X, y

(array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.19842294,
         0.        ]]),
 array([ 0.35714286,  0.11607143, -0.05      , ...,  0.        ,
         0.        ,  0.        ]))